In [43]:
import os
import sys
import torch
import torch.nn as nn
import safetensors
import functools
from safetensors.torch import save_file, save_model
from hydra import compose, initialize
import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.eval import *

## Convert to safetensors


In [ ]:
# experiment=retest trainer.devices=[4] logger.wandb.offline=true name=CLIPSeg-retest ckpt_path=/home/yl241/models/Multi-Annotators/clipseg_debris_weight.ckpt

In [40]:
@hydra.main(version_base="1.3", config_path="../configs", config_name="eval.yaml")
def main(cfg: DictConfig) -> None:
    """Main entry point for evaluation.

    :param cfg: DictConfig configuration composed by Hydra.
    """
    # apply extra utilities
    # (e.g. ask for tags if none are provided in cfg, print cfg tree, etc.)
    extras(cfg)
    evaluate(cfg)

In [44]:
    if cfg.get("seed"):
        L.seed_everything(cfg.seed, workers=True)
    if not cfg.ckpt_path:
        log.warning("No checkpoint path provided!")

    log.info(f"Instantiating datamodule <{cfg.data._target_}>")
    datamodule: LightningDataModule = hydra.utils.instantiate(cfg.data)

    log.info(f"Instantiating model <{cfg.model._target_}>")
    model: LightningModule = hydra.utils.instantiate(cfg.model)

    log.info("Instantiating callbacks...")
    callbacks: List[Callback] = instantiate_callbacks(cfg.get("callbacks"))

    log.info("Instantiating loggers...")
    logger: List[Logger] = instantiate_loggers(cfg.get("logger"))

    log.info(f"Instantiating trainer <{cfg.trainer._target_}>")
    trainer: Trainer = hydra.utils.instantiate(cfg.trainer, logger=logger, callbacks=callbacks)

    object_dict = {
        "cfg": cfg,
        "datamodule": datamodule,
        "model": model,
        "logger": logger,
        "trainer": trainer,
    }

    if logger:
        log.info("Logging hyperparameters!")
        log_hyperparameters(object_dict)

    if cfg.get("test"):
        log.info("Starting testing!")
        trainer.test(model=model, datamodule=datamodule, ckpt_path=cfg.ckpt_path)
    if cfg.get("predict"):
        log.info("Starting predictions!")
        # for predictions use trainer.predict(...)
        predictions = trainer.predict(model=model, datamodule=datamodule, ckpt_path=cfg.ckpt_path)


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


InterpolationResolutionError: ValueError raised while resolving interpolation: HydraConfig was not set
    full_key: paths.work_dir
    object_type=dict

In [39]:
cfg.paths

{'root_dir': '${oc.env:PROJECT_ROOT}', 'data_dir': '${local.data_dir}', 'log_dir': '${paths.output_dir}', 'output_dir': '${local.prediction_output_dir}/${name}', 'work_dir': '${hydra:runtime.cwd}'}

In [22]:
save_model(state_dict, "/home/yl241/models/Multi-Annotators/clipseg_debris_weight.safetensors")
print("Converted to model.safetensors")

AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'